In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset/IMDB Dataset.csv


In [4]:
import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, TFBertForSequenceClassification, create_optimizer
import string
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import joblib

2025-07-12 02:27:13.198938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752287233.431188      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752287233.500966      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
df = pd.read_csv('/kaggle/input/imdb-dataset/IMDB Dataset.csv')

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
def preprocessing(text:str) -> str:
    def lower_casing(text:str) -> str:
        return text.lower()

    def remove_html_tags(text:str) -> str:
        return re.sub(r"<[^>]+>", "", text)

    def remove_url(text:str) -> str:
        return re.sub(r"https?://\S+|www\.\S+", "", text)
    
    # def remove_punctuation(text:str) -> str:
    #     return text.translate(str.maketrans("", "", string.punctuation))
    
    # STOP_WORDS = set(stopwords.words('english'))
    # def remove_stop_words(text:str) -> str:
    #     return " ".join(word for word in text.split() if word not in STOP_WORDS)
    
    text = lower_casing(text)
    text = remove_html_tags(text)
    text = remove_url(text)
    # text = remove_punctuation(text)
    # text = remove_stop_words(text)
    return text

In [8]:
df['review'] = df['review'].apply(preprocessing)

In [9]:
le = LabelEncoder()
y = le.fit_transform(df['sentiment'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['review'].tolist(), y.tolist(), test_size=0.2, random_state = 12)

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length = 256, return_tensors = 'tf')
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length = 256, return_tensors = 'tf')

I0000 00:00:1752287394.251457      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [13]:
train = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids':tf.constant(train_encodings['input_ids']),
        'attention_mask':tf.constant(train_encodings['attention_mask'])
    },
    tf.constant(y_train)
)).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids':tf.constant(test_encodings['input_ids']),
        'attention_mask':tf.constant(test_encodings['attention_mask'])
    },
    tf.constant(y_test)
)).batch(32).prefetch(tf.data.AUTOTUNE)

In [14]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
optimizer, lr_schedule = create_optimizer(
    init_lr = 2e-5,
    num_train_steps = len(train) * 5,
    num_warmup_steps = int(0.1 * len(train) * 5),
    weight_decay_rate = 0.01
)

In [17]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(train, validation_data = (test), epochs = 5)

Epoch 1/5
1240/1240 [==============================] - 1334s 1s/step - loss: 0.7091 - accuracy: 0.6187 - val_loss: 0.6908 - val_accuracy: 0.6864
Epoch 2/5
1240/1240 [==============================] - 1297s 1s/step - loss: 0.6058 - accuracy: 0.6884 - val_loss: 0.5117 - val_accuracy: 0.7437
Epoch 3/5
1240/1240 [==============================] - 1296s 1s/step - loss: 0.5297 - accuracy: 0.7066 - val_loss: 0.6280 - val_accuracy: 0.5068
Epoch 4/5
1240/1240 [==============================] - 1297s 1s/step - loss: 0.4479 - accuracy: 0.7903 - val_loss: 0.4023 - val_accuracy: 0.9036
Epoch 5/5
1240/1240 [==============================] - 1296s 1s/step - loss: 0.3182 - accuracy: 0.9004 - val_loss: 0.3497 - val_accuracy: 0.9053


In [20]:
model.save_pretrained('/kaggle/working/bert-sentiment-analysis')
tokenizer.save_pretrained('/kaggle/working/bert-sentiment-tokenizer')

Provided path (/kaggle/working/bert-sentiment-analysis) should be a directory, not a file


('/kaggle/working/bert-sentiment-tokenizer/tokenizer_config.json',
 '/kaggle/working/bert-sentiment-tokenizer/special_tokens_map.json',
 '/kaggle/working/bert-sentiment-tokenizer/vocab.txt',
 '/kaggle/working/bert-sentiment-tokenizer/added_tokens.json')